In [ ]:
!pip install pandas

#from google.colab import drive
import os
import pandas as pd

# !pip install --upgrade fastai

#drive.mount('/content/drive')

In [ ]:
from pathlib import Path
ratings_dir = Path("My Drive/deep_learning_training/experiments/letterboxd/crawled/ratings/")

df_all = pd.read_csv(ratings_dir / "special.csv")

df_all = pd.concat(
    [df_all]
    +
    [ pd.read_csv(f) for f in ratings_dir.glob("*.csv") if f.stem.isdigit()],
    ignore_index=True
)

len(df_all)

19621514

In [9]:
import psutil
ram = psutil.virtual_memory()
print(f"Total RAM: {ram.total / 1024**3:.2f} GB")
print(f"Available RAM: {ram.available / 1024**3:.2f} GB")
print(f"Used RAM: {ram.used / 1024**3:.2f} GB")
print(f"Percentage Used: {ram.percent}%")

df_all.info(memory_usage="deep")



Total RAM: 15.94 GB
Available RAM: 5.67 GB
Used RAM: 10.27 GB
Percentage Used: 64.4%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621514 entries, 0 to 19621513
Data columns (total 3 columns):
 #   Column    Dtype  
---  ------    -----  
 0   user_id   object 
 1   movie_id  object 
 2   rating    float64
dtypes: float64(1), object(2)
memory usage: 1.7 GB


In [12]:
print("Total ratings size: ", len(df_all))

movie_popularity = df_all["movie_id"].value_counts()
popular_movies = movie_popularity[movie_popularity >= 100].index
df = df_all[df_all["movie_id"].isin(popular_movies)]
print("Ratings size after movie filtering: ", len(df))

unique_users = df["user_id"].unique()
sampled_users = unique_users[0:5000]
df = df[df["user_id"].isin(sampled_users)]
print("Ratings size after user filtering: ", len(df))

Total ratings size:  19621514


MemoryError: Unable to allocate 126. MiB for an array with shape (16479530,) and data type int64

In [ ]:
from fastai.collab import CollabDataLoaders

dls = CollabDataLoaders.from_df(df, bs=64)
dls.show_batch()

,user_id,movie_id,rating
0,livingdeadyke,frozen-2013,4.0
1,cinemonster,chinatown,5.0
2,gregorkreyca,x-men-days-of-future-past,4.0
3,kayleyy_lc,noelle-2019,2.5
4,belial_carboni,pg-psycho-goreman,4.0
5,jryn7,the-burrowers,3.0
6,michael11391,a-bronx-tale,4.5
7,skipper_schutt,joy-2015,2.5
8,eldeibik,the-meg,2.0
9,dude__0g,the-big-lebowski,5.0


In [ ]:
from fastai.collab import collab_learner
learn = collab_learner(dls, n_factors=150, y_range=(0, 5.5))
learn.fit_one_cycle(8, lr_max=0.001, wd=0.003)

epoch,train_loss,valid_loss,time


KeyboardInterrupt: 

In [ ]:
import torch
def sigmoid_range(x, bounds=(0, 1)):
    low, high = bounds
    return low + (high - low) * torch.sigmoid(x)

def predict_ratings(user_name):
  user_id = dls.classes['user_id'].o2i[user_name]
  watched_movies = df[df['user_id'] == user_name]['movie_id'].astype(str).to_list()
  predicted_movies = [x for x in dls.classes['movie_id'] if x not in watched_movies]
  #predicted_movies = [x for x in dls.classes['movie_id']]

  prediction_list = []
  for movie_name in predicted_movies:
    movie_id = dls.classes['movie_id'].o2i[movie_name]
    dot = torch.dot(learn.model.u_weight.weight[user_id], learn.model.i_weight.weight[movie_id])
    ubias = learn.model.u_bias.weight[user_id].squeeze()
    ibias = learn.model.i_bias.weight[movie_id].squeeze()
    dot += ubias + ibias
    dot = sigmoid_range(dot, (0,5.5))
    prediction_list += [(movie_name, dot.item())]

  prediction_list = sorted(prediction_list, key=lambda x: x[1], reverse=True)
  return prediction_list[:20]

predict_ratings("drakman")

[('psycho', 4.390081882476807),
 ('in-the-mood-for-love', 4.320937156677246),
 ('eyes-wide-shut', 4.200275897979736),
 ('before-sunset', 4.193432807922363),
 ('vertigo', 4.170406818389893),
 ('blue-velvet', 4.142148017883301),
 ('before-sunrise', 4.1349778175354),
 ('dead-poets-society', 4.065926551818848),
 ('scream', 3.9738478660583496),
 ('the-texas-chain-saw-massacre', 3.9697697162628174),
 ('the-iron-claw-2023', 3.9643445014953613),
 ('sinners-2025', 3.944124460220337),
 ('brokeback-mountain', 3.922883987426758),
 ('little-miss-sunshine', 3.9119462966918945),
 ('the-exorcist', 3.873560667037964),
 ('suspiria', 3.807309627532959),
 ('frances-ha', 3.785313367843628),
 ('moonrise-kingdom', 3.781148672103882),
 ('pearl-2022', 3.7622201442718506),
 ('licorice-pizza', 3.7354679107666016)]

In [ ]:
learn.save('movie_50+_user_500_embeddings_100.pkl')

Path('models/movie_50+_user_500_embeddings_100.pkl.pth')